# 6. Объединение DataFrame: concat
Следуя нашему плану объединения таблиц, первым делом мы должны склеить таблицы ratings1 и ratings2 по строкам.

Для этого воспользуемся встроенной функцией Pandas [concat()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html), которая позволяет склеивать (конкатенировать) таблицы как по строкам, так и по столбцам.

Кликните на плашку, чтобы увидеть информацию ↓

Основные параметры функции concat():
- objs — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;
- axis — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;
- join — либо inner (пересечение), либо outer (объединение); рассмотрим этот момент немного позже;
- ignore_index — по умолчанию установлено значение False, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение True, параметр будет игнорировать исходные значения и повторно назначать значения индекса в последовательном порядке.
Для корректной конкатенации по строкам объединяемые таблицы должны иметь одинаковую структуру — идентичное число и имена столбцов.

Итак, давайте склеим  ratings1 и ratings2 по строкам, так как они имеют одинаковую структуру столбцов. Для этого передадим их списком в функцию concat(). Помним, что параметр axis по умолчанию равен 0, объединение происходит по строкам, поэтому не трогаем его. 

Примечание. Обратите внимание, что concat является функцией библиотеки, а не методом DataFrame. Поэтому её вызов осуществляется как pd.concat(...).

In [1]:
import pandas as pd


ratings1 = pd.read_csv('data/ratings1.csv', sep=',')
ratings2 = pd.read_csv('data/ratings2.csv', sep=',')

ratings = pd.concat([ratings1, ratings2])
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
60831,610,166534,4.0
60832,610,168248,5.0
60833,610,168250,5.0
60834,610,168252,5.0


В результате мы увеличили первую таблицу, добавив снизу строки второй таблицы.

На первый взгляд может показаться, что всё прошло успешно, однако если мы посмотрим на индексы последних строк таблицы, то увидим, что их нумерация не совпадает с количеством строк. Это может привести к некорректному объединению таблиц по ключевым столбцам на следующем этапе решения нашей задачи.

Это связано с тем, что по умолчанию concat сохраняет первоначальные индексы объединяемых таблиц, а обе наши таблицы индексировались, начиная от 0. Чтобы создать новые индексы, нужно выставить параметр ignore_index на True:

In [2]:
ratings = pd.concat([ratings1, ratings2], ignore_index=True)
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


Казалось бы, совсем другое дело! Но это ещё не всё. Давайте узнаем количество строк в таблицах ratings и dates, ведь нам предстоит вертикально склеить их между собой:

In [3]:
dates = pd.read_csv('data/dates.csv', sep=',')

print(f'Число строк в таблице ratings: {ratings.shape[0]}')
print(f'Число строк в таблице dates: {dates.shape[0]}')


Число строк в таблице ratings: 100837
Число строк в таблице dates: 100836


Размерность таблиц разная — как такое могло произойти?

На самом деле очень просто: при выгрузке данных информация об оценках какого-то  пользователя попала в обе таблицы (ratings1 и ratings2). В результате конкатенации случилось дублирование строк. В данном примере их легко найти — выведем последнюю строку таблицы ratings1 и первую строку таблицы ratings2:

In [4]:
ratings1.tail(1)

,userId,movieId,rating
40000,274,5621,2.0


In [5]:
ratings2.head(1)

,userId,movieId,rating
0,274,5621,2.0


Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame drop_duplicates(), который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, выставив параметр ignore_index в методе drop_duplicates() на значение True:

In [6]:
ratings = ratings.drop_duplicates(ignore_index=True)
print(f'Число строк в таблице ratings: {ratings.shape[0]}')

Число строк в таблице ratings: 100836


Наконец, мы можем добавить к нашей таблице с оценками даты их выставления. Для этого конкатенируем таблицы ratings и dates по столбцам:

In [7]:
ratings_dates = pd.concat([ratings, dates], axis=1)
ratings_dates.tail(7)

,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


Сохраним rating_dates для следующего модуля:

In [8]:
ratings_dates.to_csv('./data/ratings_dates.csv')

✍ Итак, мы смогли создать единую таблицу с рейтингами и датами их представления. Нашим следующим шагом будет присоединить к таблице информацию о фильмах из таблицы movies.

А пока предлагаем вам потренироваться в использовании функции concat() ↓

### Задание 6.1
Какой параметр функции concat позволяет управлять способом конкатенации (проводить конкатенацию по строкам или по столбцам)?
- axis
- join
- keys
- ignore_index

Овтет: axis

### Задание 6.2
Заданы две таблицы — df1 и df2. В первой содержатся имена и фамилии сотрудников, во второй — их должности.

In [9]:
df1 = pd.DataFrame({"Name": ["Pankaj", "Lisa"],
                   "Surname": ["Sobolev", "Krasnova"]})
df2 = pd.DataFrame({"Role": ["Admin", "Editor"]})


Какой из приведённых ниже способов будет верным при объединении этих таблиц?
- df = pd.concat([df1, df2])
- df = pd.concat([df1, df2], axis=1)
- df = pd.concat([df1, df2], ignore_index=True)
- df = pd.concat([df2, df1)

In [10]:
df = pd.concat([df1, df2], axis=1)
df

,Name,Surname,Role
0,Pankaj,Sobolev,Admin
1,Lisa,Krasnova,Editor


ЗАДАНИЕ 6.3 (ВЫПОЛНЯЕТСЯ В CODEBOARD НИЖЕ)

Допустим, в ваше распоряжение предоставлена директория "./Root/users/". В данной директории содержатся csv-файлы, в каждом из которых хранится информация об идентификаторах пользователей (user_id) и ссылки на их фотографии (photo_url). Каждый файл из папки users имеет примерно следующую структуру:

![](./img/dst3-u1-md12_6_6.png)

При проверке в директории может быть сколько угодно файлов (директория может изменяться в зависимости от устройства файловой системы).

Вам необходимо написать функцию concat_user_files(path), параметром которой является path — путь до директории. Функция должна объединить информацию из предоставленных вам файлов в один DataFrame и вернуть его. 

Список названий всех файлов, находящихся в директории, вы можете получить с помощью функции [os.listdir(path)](https://docs-python.ru/standart-library/modul-os-python/funktsija-listdir-modulja-os/) из модуля os (модуль уже импортирован в файле main.py). Например, для директории "./Root/users/" результатом работы функции будет список:
```python
print(os.listdir('./Root/users/'))
['users2.csv', 'users1.csv', 'users3.csv']
```
Примечание. Модуль os позволяет работать с операционной системой компьютера прямо из кода. Подробнее о нем вы можете почитать [здесь](https://pythonworld.ru/moduli/modul-os.html).

Отсортируйте этот список, прежде чем производить объединение файлов.

Когда вы получите отсортированный список, вам останется только прочитать все csv-файлы из списка в цикле и объединить прочитанные таблицы между собой.

Однако обратите внимание, что метод os.listdir() возвращает только список имён файлов в указанной директории, а при чтении файла необходимо указывать полный путь до него. То есть путь для чтения будет таким:
```python
'./Root/users/{file_name}'
```
Не забудьте обновить индексы результирующей таблицы после объединения.

Учтите, что на тестовом наборе файлов в результате объединения могут возникнуть дубликаты, от которых необходимо будет избавиться.

Например, для директории "./Root/users/" результирующая таблица должна иметь следующий вид:

![](./img/dst3-u1-md12_6_6.png)

При работе с заданием в Codeboard не используйте кириллицу — писать решение надо полностью на английском языке, иначе написанный вами код не запустится!

Начните работу традиционно с открытия файла main.py — там вы увидите условие задачи.
Впишите требуемый код и нажмите RUN.
Нажмите SUBMIT, чтобы отправить ваш ответ на проверку и получить баллы в случае верного решения.

✏️Дисклеймер: Обратите внимание, что по причине более старой версии pandas (0.13.1) в Codeboard метод drop_duplicates() не имеет аргумента ignore_index (он появился в версии 1.0.0). Мы работаем над данной проблемой.

In [11]:
import os

path = './Root/users'
filenames = sorted(os.listdir(path))
filepaths = [f'{path}/{filename}' for filename in filenames]
users_dfs = [pd.read_csv(filepath, sep=',')  for filepath in filepaths]

users = pd.concat(users_dfs)
users = users.drop_duplicates()
new_index = list(range(users.shape[0]))
users.index = new_index
users

,user_id,image_url
0,id001,http://example.com/img/id001.png
1,id002,http://example.com/img/id002.jpg
2,id003,http://example.com/img/id003.bmp
3,id004,http://example.com/img/id004.jpg
4,id005,http://example.com/img/id005.png
5,id006,http://example.com/img/id006.png
6,id007,http://example.com/img/id007.png
7,id008,http://example.com/img/id008.png
